In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


In [5]:
starbucks_response = requests.get("https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state")

In [6]:
starbucks_soup = BeautifulSoup(starbucks_response.content, "html.parser")

In [12]:
table = starbucks_soup.find_all("tbody", {"class": "relative z-10 text-sm"})

<table class="wpr-table"><thead class="table-head bg-wpr-table_header_bg relative z-40 px-0 py-0 text-xs uppercase tracking-wider"><tr><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 pin left-0 z-50 border-l-0"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">State</div></div></th><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 z-40"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">Starbucks Stores 2023<svg class="icon icon-tabler icon-tabler-chevron-down" fill="none" height="14" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" viewbox="0 0 24

In [13]:
# initialize an empty list
rows = []

# iterate over all rows in the population table
for state in table:

    # Get all the cells (<td>) in the row.
    cells = state.find_all("tr")


    # Find the the name of the course in cell[0]
    name_tag = cells[0]
    name = name_tag.get_text(strip = True)

    # Find the term the course is offered in cell[1]
    # which for most of the courses is contained in the <a> tag
    link =  cells[1]
    num_loc = link.get_text(strip = True)

    # Append this data.
    rows.append({
        "State": name,
        "Starbucks Locations": num_loc

})

IndexError: list index out of range

In [14]:
pd.DataFrame(rows)

""


In [16]:
table = (starbucks_soup.find_all("table"))[0]
table

<table class="wpr-table"><thead class="table-head bg-wpr-table_header_bg relative z-40 px-0 py-0 text-xs uppercase tracking-wider"><tr><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 pin left-0 z-50 border-l-0"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">State</div></div></th><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 z-40"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">Starbucks Stores 2023<svg class="icon icon-tabler icon-tabler-chevron-down" fill="none" height="14" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" viewbox="0 0 24

In [17]:
starbs= []

# iterate over all rows in the population table
for row in table.find_all("tr")[1:]:

  #state
  state= row.find("th").text

  #num starbs
  num=row.find("td").text

  starbs.append({
                "state": state,
                "Starbucks Stores": num,
  })

In [19]:
pd.DataFrame(starbs)

,state,Starbucks Stores
0,California,"3,080"
1,Texas,"1,346"
2,Florida,844
3,Washington,741
4,New York,692
5,Illinois,677
6,Arizona,548
7,Colorado,495
8,Ohio,491
9,Virginia,489
